# Calculating implied infection numbers

This notebook tries to compute what the full infection numbers in the past and present likely were/are.

It does so in the past by blending variables for "median days from infection to death" and "infection fatility rate" (IFR) with smoothed death rates. In other words, days_to_death days before date D, there must have been roughly (deaths_on_date_D / IFR) infections to end up with a given number of deaths on date D.

It does in the present to looking at what percentage of infections were confirmed on the last day calculated in the past, and applying that percentage to the new infections found since then. That doesn't quite take into account if there is a significant ramping of testing during that time, but it should be close enough.

The principal source of death data is files from the NY Times, supplemented by a more accurate DateOfDeath.csv from Massachusetts. The source of testing data is The COVID Tracking Project, maintained by The Atlantic.

NOTE: Prior to running this notebook, you should retrieve the latest DateOfDeath.csv file by:

1. going to https://www.mass.gov/info-details/covid-19-response-reporting,
2. downloading the raw data zip from the line saying "Raw data used to create the dashboard is available here:"
3. copying the DateofDeath.csv in that file to the same directory as the notebook

Yeah, that could be automated. Just haven't done it yet...

In [ ]:
%matplotlib inline
import numpy
import pandas
import matplotlib
import matplotlib.pyplot as plt

from common import load_data

In [ ]:
# Earliest date that there is sufficient data for all states, including MA
EARLIEST_DATE = pandas.Period('2020-03-10', freq='D')
LATEST_DATE = None
LATEST_DATE = pandas.Period('2020-07-29', freq='D')


In [ ]:
meta, nyt_stats, ct_stats = load_data(EARLIEST_DATE, LATEST_DATE)

### Put the two datasets together

In [ ]:
ct1 = ct_stats.set_index(['ST', 'Date']).sort_index()[['Pos']]
nyt1 = nyt_stats.set_index(['ST', 'Date']).sort_index()[['Deaths']]
stats = ct1.join(nyt1)
stats.head()

### Calculate new stats, state by state

In [ ]:
def smooth_series(s):
    i = 0
    last_i = None
    last_val = None
    run_length = 0
    foo = s.copy()
    while i < len(foo):
        val = foo[i]
        if pandas.isna(val):
            last_i, last_val = i, None
            run_length = 0
        elif last_val is None:
            last_i, last_val = i, val
            run_length = 1
        elif val == last_val:
            run_length += 1
        #elif (val == (last_val + 1)) or (run_length == 1):
        elif run_length == 1:
            last_i, last_val = i, val
            run_length = 1
        else:
            run_length += 1
            new_vals = numpy.linspace(last_val, val, run_length)
            foo[last_i:i+1] = new_vals
            last_i, last_val = i, val
            run_length = 1
        i += 1

    return foo

In [ ]:
def calc_state_stats(state, state_stats, meta):
    st = state_stats.groupby('Date').sum().sort_index().copy()

    st['ST'] = state
    st['Pop'] = meta.loc[state].Pop

    # Prep for 7-day smoothing calculations
    st7 = st.shift(7)
    
    # Smooth series that might not be reported daily in some states
    st.Pos = smooth_series(st.Pos)
    st.Deaths = smooth_series(st.Deaths)

    # Calculate daily confirmed positive cases and deaths
    st['Confirms'] = (st.Pos - st.shift().Pos)
    st['Daily'] = (st.Deaths - st.shift().Deaths)

    # Calculate both as trailing 7-day averages
    st['Confirms7'] = (st.Pos - st7.Pos) / 7
    st['Deaths7'] = (st.Deaths - st7.Deaths) / 7

    # Convert into central 7-day mean, with special handling for last three days
    #st.Confirms7 = st.Confirms7.shift(-3)
    #st.Deaths7 = st.Deaths7.shift(-3)
    specs = (
        (8., numpy.array([1., 1., 1.1, 1.2, 1.4, 1.2, 1.1])),
        (9., numpy.array([1., 1.1, 1.1, 1.2, 1.4, 1.7, 1.5])),
        (10., numpy.array([1., 1.1, 1.2, 1.3, 1.4, 1.8, 2.2])),
    )
    for col in [-2, -1]:
        st.iloc[:, col] = st.iloc[:, col].shift(-3)
        dailies = st.iloc[-7:, col-2].values
        vals = [((dailies * factors).sum() / divisor) for divisor, factors in specs]
        st.iloc[-3:, col] = vals

    return st.reset_index().set_index(['ST', 'Date']).copy()

In [ ]:
meta_tmp = meta.set_index('ST')[['Pop']]

In [ ]:
states = [calc_state_stats(state, df, meta_tmp)
          for state, df in stats.reset_index().groupby('ST')]
states[-17].tail()

In [ ]:
def get_infections_df(states, death_lag, ifr_high, ifr_low, incubation, infectious):
    new_states = []
    for state in states:
        state = state.copy()

        # Calculate the IFR to apply for each day
        ifr = pandas.Series(numpy.linspace(ifr_high, ifr_low, len(state)), index=state.index)
        # Calculate the infections in the past
        infections = state.shift(-death_lag).Deaths7 / ifr

        # Find out the ratio of infections that were detected on the last date in the past
        last_date = infections.index[-(death_lag+1)]
        last_ratio = infections.loc[last_date] / (state.loc[last_date, 'Confirms7'] + 1)

        # Apply that ratio to the dates since that date
        infections.iloc[-death_lag:] = state.Confirms7.iloc[-death_lag:] * last_ratio

        state['NewInfections'] = infections
        state['TotInfections'] = infections.cumsum()
        state['ActiveInfections'] = infections.rolling(infectious).sum().shift(incubation)
        state['ActiveKnown'] = state.Confirms7.rolling(infectious).sum()
        state['ActiveUnknown'] = state.ActiveInfections - state.ActiveKnown
        state['APer1000'] = state.ActiveInfections / state.Pop / 1000.
        state['AUPer1000'] = state.ActiveUnknown / state.Pop / 1000.
        new_states.append(state)

    return pandas.concat(new_states)

In [ ]:
infected_states = get_infections_df(states, 18, 0.01, 0.006, 4, 13)
infected_states.loc['WV', 'Deaths'].tail()

In [ ]:
foo = infected_states.reset_index().set_index(['Date', 'ST']).sort_index()
foo = foo[['Pop', 'Confirms7', 'Deaths7', 'NewInfections', 'APer1000', 'AUPer1000']]
foo['PctFound'] = foo.Confirms7 / (foo.NewInfections + 1)

In [ ]:
faz = foo.loc['2020-07-27', :].sort_values('AUPer1000', ascending=False).copy()
faz = faz.reset_index()[['ST', 'Pop', 'Confirms7', 'Deaths7', 'AUPer1000', 'PctFound']]
faz.columns = ['ST', 'Pop', 'Cases', 'Deaths', 'ActUnk1000', 'PctFound']
faz.set_index('ST').head(12)

In [ ]:
fam = foo.reset_index()
fam = fam[fam.ST == 'NJ']
fam.set_index('Date').tail(20)

In [ ]:
SCENARIOS = (('1.0% - 0.6%', 20, 0.01, 0.006), )

In [ ]:
raise ValueError()

## Now for the charts...

In [ ]:
SCENARIOS = (('20', 20, 0.01, 0.01), ('18', 18, 0.01, 0.01), ('16', 16, 0.01, 0.01), )

df = get_infections_df(SCENARIOS)
foo = df.plot(title="New Infections Estimates, varying average days to death, IFR = 1.0%", figsize=(10,5))

In [ ]:
SCENARIOS = (('1.3%', 18, 0.013, 0.013), ('1.0%', 18, 0.01, 0.01), ('0.7%', 18, 0.007, 0.007), )

df = get_infections_df(SCENARIOS)
foo = df.plot(title="New Infections Estimates, varying IFR, days to death = 18", figsize=(10,5))

In [ ]:
SCENARIOS = (('1.2% - 0.8%', 18, 0.012, 0.008), ('1.0% - 0.7%', 18, 0.01, 0.007), ('0.9% - 0.6%', 18, 0.009, 0.006), )

df = get_infections_df(SCENARIOS)
foo = df.plot(title="Infection Estimations, improving IFR, days to death = 18", figsize=(10,5))

In [ ]:
SCENARIOS = (('1.0% - 0.6%', 20, 0.01, 0.006), )

df = get_infections_df(SCENARIOS)
foo = df.plot(title="Infection Estimations, my hunch, 20 median days to death", figsize=(10,5))

In [ ]:
df.sum()

In [ ]:
df.tail()

In [ ]:
SCENARIOS = (('1.2% - 0.5%', 21, 0.012, 0.005), )

df = get_infections_df(SCENARIOS)
foo = df.plot(title="Worst case? 21 days to death, improving IFR", figsize=(10,5))

In [ ]:
numpy.linspace(1, 2, 3)